# Milestone 1

### Importing Needed Libraries

In [ ]:
import pdfplumber 
import fitz 
import os 
import re 
import json 
import pandas as pd
import nltk 
import matplotlib.pyplot as plt
import seaborn as sns
import nlp
from wordcloud import WordCloud
from collections import Counter
from langchain.text_splitter import RecursiveCharacterTextSplitter
import spacy
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')


### Connecting the code with the files in PDF folder in the repo

In [ ]:
pdf_dir = 'PDF files/'
pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith('.pdf')]

### Defining function for Extraction , Preprocessing , tokenizing and removing stop words

In [ ]:
#Text Extraction
def extract_text (pdf_path ):
    text = ''
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text("text") + "\n"
    return text.strip()

#Cleaning and removing noise
def clean_text(text):
    # Remove extra spaces and new lines, but keep punctuation like periods and question marks
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9.?! ]', '', text)  # Keep important punctuation
    return text.lower().strip()

def chunk_text(text, chunk_size=500, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,  # Max characters per chunk
        chunk_overlap=chunk_overlap,  # Overlap for context continuity
        length_function=len,  # Measure length by characters
        separators=["\n\n", "\n", ".", " ", ""]  # Hierarchical splitting
    )
    return text_splitter.split_text(text) 

# Tokenizing the text into sentences
def tokenize_text(text):
    return [s.strip() for s in sent_tokenize(text) if s.strip()]


#removing stop words in each sentence
def removing_stopwords(sentences):
    stop_words = set(stopwords.words('english'))
    filtered_sentences = []

    for sentence in sentences:
        words = word_tokenize(sentence)  
        filtered_words = [word for word in words if word.lower() not in stop_words]  
        filtered_sentences.append(' '.join(filtered_words))  

    return filtered_sentences

### Listing all availabe uploaded PDF files and the user will input the number of file he wants 
### Note : will be changed later so the user can upload his own pdf file

In [ ]:
print('\n📂 Available PDFs:')
for i, pdf in enumerate(pdf_files,1):
    print(f'{i} : {pdf}')
choice = int(input("\nEnter the number of the PDF you want to extract text from: ")) - 1
if 0 <= choice < len(pdf_files):
    PDF_FILE = pdf_files[choice]
    PDF_PATH = os.path.join(pdf_dir, PDF_FILE)
    print(f"\n✅ Extracting text from: {PDF_FILE}")
    text = extract_text(PDF_PATH)
    print(f'\n {text}')


### Preprocessing the text

In [ ]:
cleaned_text = clean_text(text)
tokenized_text = tokenize_text(cleaned_text)
filtered_text = ' '.join(removing_stopwords(tokenized_text))
chunks = chunk_text(cleaned_text)


### Basic Statistics

In [ ]:
words = word_tokenize(filtered_text)

total_words = len(words)
unique_words = len(set(words))
total_sent = len(tokenized_text)
print("\n📊 Basic Text Statistics:")
print(f"🔹 Total Words: {total_words}")
print(f"🔹 Unique Words: {unique_words}")
print(f"🔹 Total Sentences: {total_sent}")

## Visualization Time

### Most frequent 20 word

In [ ]:
words = [word.lower() for word in words if word.isalnum()]  
words_count = Counter(words)
most_common_words = words_count.most_common(20)
plt.figure(figsize=(12, 6))
sns.barplot(x=[word for word, _ in most_common_words], y=[count for _, count in most_common_words])
plt.xticks(rotation=45)
plt.xlabel("Words")
plt.ylabel("Frequency")
plt.title("Top 20 Most Common Words")
plt.show()

### WordCloud

In [ ]:
wordcloud = WordCloud(width=800, height=400, background_color="white").generate(filtered_text)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud")
plt.show()

## Saving extracted text into .json file

In [ ]:
def save_to_json(data,filename):
    if not os.path.exists('Extracted text'):
        os.makedirs('Extracted text')
    
    file_path = os.path.join('Extracted text', filename)  
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
    
    print(f"✅ Filtered text saved as {file_path}")

In [ ]:
save_to_json(filtered_text, 'Art of War') #Example 

# Milestone 2

### importing the needed libraries SentenceTransformer and faiss ###

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

### Choosing the embedding model ###

In [ ]:
def generate_embeddings(chunks, model_name='all-MiniLM-L6-v2', batch_size=32):
    """
    Generate embeddings for text chunks using SentenceTransformer.
    """
    model = SentenceTransformer(model_name)
    embeddings = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]
        batch_embeddings = model.encode(batch, convert_to_numpy=True, show_progress_bar=True)
        embeddings.append(batch_embeddings)
    return np.vstack(embeddings), model

# Creating FAISS Index

After generating embeddings for the sentences, we use FAISS (Facebook AI Similarity Search) to create an index that stores these embeddings. The index allows for fast similarity searches using the L2 (Euclidean) distance metric.

In [ ]:
def setup_faiss_index(embeddings, index_file='faiss_index.bin'):
    """
    Set up a FAISS index for storing embeddings and save it to disk.
    """
    try:
        dimension = embeddings.shape[1]
        index = faiss.IndexFlatL2(dimension)  # L2 distance for similarity search
        index.add(embeddings)
        faiss.write_index(index, index_file)
        print(f"✅ FAISS index saved to {index_file}")
        return index
    except Exception as e:
        print(f"Error setting up FAISS index: {e}")
        return None

In [ ]:
index_file = f'faiss_index_{PDF_FILE}.bin'
embeddings, model = generate_embeddings(chunks)
faiss_index = setup_faiss_index(embeddings, index_file)

# Performing the Search

For a given query, we generate its embedding using the SentenceTransformer model and search for the most similar sentences in the FAISS index. The top-k most relevant sentences are then returned.

In [ ]:
def search_similar_faiss(query, chunks, index, model, top_k=5, threshold=1.0):
    query_embedding = model.encode([query], convert_to_numpy=True)
    D, I = index.search(query_embedding, top_k)  # D: distances, I: indices
    results = [(chunks[i], D[0][j]) for j, i in enumerate(I[0]) if D[0][j] <= threshold]
    return results

# Query Preprocessing

The user's query is processed in the same way as the document text: cleaned, tokenized, and stopwords are removed. This ensures that the query is in the same format as the sentences stored in the index.

In [ ]:
def preprocess_query(query):
    # Clean and remove stop words from the query, similar to your text preprocessing
    cleaned_query = clean_text(query)
    tokenized_query = tokenize_text(cleaned_query)
    filtered_query = ' '.join(removing_stopwords(tokenized_query))
    return filtered_query

# User Query Input

The user enters a search query, which is then processed and used to retrieve the most relevant sentences from the document.

In [ ]:
test_query = input("\nEnter a test query (e.g., 'What is the art of war about?'): ")
print("\n🔍 Top relevant chunks:")
results = search_similar_faiss(test_query, chunks, faiss_index, model, top_k=5)
for i, (chunk, distance) in enumerate(results, 1):
    print(f"Chunk {i}: {chunk[:100]}... (L2 Distance: {distance:.4f})")
